In [1]:
#importing library
from bs4 import BeautifulSoup,NavigableString, Comment
import requests
import time
import re
from datetime import datetime
import html5lib



In [22]:

#importing library
from bs4 import BeautifulSoup,NavigableString, Comment
import requests
import time
import re
from datetime import datetime
import html5lib
def get_content(url):
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,  'html.parser')
    try:
        #remove the last 4 element
        article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
        h2 = article.find('h2')
        h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
        # Update the text of the h2 tag
        h2.string = h2_text
        
        for script_or_style in article(['script', 'style']):
            script_or_style.decompose()
        article.find('div',recursive = True, class_ = 'bv-lq').decompose()
        article.find('p', class_ = 'linkOrigin').decompose()
        caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
        """for i in article.find_all('p', recursive = True):
            if i.get('style') == 'text-align: center;':
                i.decompose()"""
        #find a,span and remove tag with text
        tags_to_remove = article.find_all(['a', 'span'])
        for tag in tags_to_remove:
            if tag.text !=  '':
                # Extract the text from the tag
                tag_text = tag.get_text()
                # Replace the tag with its text content
                tag.replace_with(tag_text)
                tag.text.strip()
        # set attribut for img
        b = article.find_all('img')
        for i in b:
            if '.svg' in i['src']:
                i.decompose()
        #for i in article.find_all('img'):
            #if 'data-original' in i.attrs.keys():
                #i['src'] = i.get('data-original')
            #i.attrs = ['class', 'alt', 'src', 'data-original']
            #del i['onclick']
            #del i['style']
            #del i['class']
        list_attr = ['src','alt','data-src', 'data-original']
        for i in article.find_all('img'):
            for j in list(i.attrs.keys()):
                if j not in list_attr:
                    i.attrs.pop(j)
        for i in article.find_all('img'):
            if 'https://image-us.24h.com.vn/' not in i['src']:
                try:
                    i['src']= i['data-original']
                except KeyError:
                    continue
        img_list = article.find_all('img')
        n_img = len(img_list)
        #print(len(caption_text_list))
        for i in range(0,n_img):
            try:
                # Create NavigableString objects for the custom caption tags
                caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
                caption_end = NavigableString("[/caption]")
                try:
                    caption_text = NavigableString(caption_text_list[i].text.strip())
                except IndexError:
                    caption_text = ''
                # Insert the caption start tag before the <img> tag
                img_list[i].insert_before(caption_start)
                # Insert the caption end tag after the <img> tag
                img_list[i].insert_after(caption_end)
                img_list[i].insert_after(caption_text)
                # Wrap the entire structure (caption start, img, caption end) in a new tag if needed
                # For example, wrapping in a <div>
                new_div = soup.new_tag("div")
                caption_start.wrap(new_div)
                caption_text_list[i].decompose()
            except IndexError as e:
                print(e)
        for i in article.find_all(recursive = True):
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
                
            except TypeError:
                continue
        for i in article.find_all('img'):
            i['class'] = "aligncenter"
            i['width'] = 800
            i['height'] = 400
        source_tag = soup.new_tag('i') 
        source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
        #remove empty div or empty space from element
        #for i in article.find_all('div',recursive= True):
            #if i.string == None:
                #if i.text =='':
                    #i.decompose()
        for element in article.find_all(recursive = True,string=True):
            if isinstance(element, NavigableString) and element.strip() == '':
                element.extract()
        #remove html comment from element   
        for comment in article.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()
        # Append the <i> tag as the last child of the <article> tag
        article.append(source_tag)
    except (AttributeError, IndexError, TypeError):
        try:
            article = soup.find('div', id= 'magazine_news')
            videos = article.find_all('div', class_ = 'videoUpload24h')
            vd_captions = article.find_all('div', class_ = 'mg-video-caption')
            for vd_caption in vd_captions:
                vd_caption.decompose()
            for video in videos:
                video.decompose()
            tags_to_remove = article.find_all(['a', 'span'])
            for tag in tags_to_remove:
                # Extract the text from the tag
                tag_text = tag.get_text()
                # Replace the tag with its text content
                tag.replace_with(tag_text)
                tag.text.strip()
            # set attribut for img
            b = article.find_all('img')
            for i in b:
                if '.svg' in i['src']:
                    i.decompose()
            b = article.find_all('img')
            for i in article.find_all('img'):
                try:
                    if 'imgFullMobile_chuan' in  i.parent['class'] or 'imgFullMobile' in  i.parent['class']:
                        i.decompose()
                except KeyError as e:
                    print(e)   
            for i in article.find_all('img'):
                if 'data-original' in i.attrs.keys():
                    i['src'] = i.get('data-original')
                #i.attrs = ['class', 'alt', 'src', 'data-original']
                #del i['onclick']
                #del i['style']
                #del i['class']
                list_attr = ['src','alt','data-src','data-original']
                for i in article.find_all('img'):
                    for j in list(i.attrs.keys()):
                        if j not in list_attr:
                            i.attrs.pop(j)
                for i in article.find_all('img'):
                    if 'https://image-us.24h.com.vn/' not in i['src']:
                        try:
                            i['src']= i['data-original']
                        except KeyError:
                            continue
            for script_or_style in article(['script', 'style']):
                script_or_style.decompose()
            for i in article.find_all(recursive = True):
                try:
                    del i['onclick']
                    del i['id']
                    del i['class']
                    del i['style']
                except AttributeError:
                    continue
                    
                except TypeError:
                    continue
            for i in article.find_all('img'):
                i['class'] = "aligncenter"
                i['width'] = 800
                i['height'] = 400
            source_tag = soup.new_tag('i') 
            source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
            #remove empty div or empty space from element
            #for i in article.find_all('div',recursive= True):
                #if i.string == None:
                    #if i.text =='':
                        #i.decompose()
            for element in article.find_all(recursive = True,string=True):
                if isinstance(element, NavigableString) and element.strip() == '':
                    element.extract()
            #remove html comment from element   
            for comment in article.find_all(string=lambda text: isinstance(text, Comment)):
                comment.extract()
            # Append the <i> tag as the last child of the <article> tag
            article.append(source_tag)
        except AttributeError as e:
            print(e)
    return article
#convert time from post to the format of output
def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
#Convert time from post to datetime format
def convert_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''
def get_post(url):
    try:
        response = requests.get(url)
        time.sleep(3)
        soup = BeautifulSoup(response.content, 'html5lib')
        content = get_content(url)
        post_time = soup.find('time').text.strip()
        published_date = convert_string(post_time)
        title = soup.find('h1').text.strip()
        #h2 = soup.find('h2').text.strip()
        #images_src = [i.attrs['src'] for i in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('img')[:-1] if 'svg' not in i.attrs['src']]
        #images_src = [img['data-original'] if 'https://image-us.24h.com.vn' not in img['src'] else img['src'] if 'svg' not in img['src'] else '' for img in soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find_all('img')[:-1]]
        #text_list = [ child.text for child in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('p')[:-3] if re.sub(r'\n+', '', child.text) != ""]
        #text_list = [h2] + text_list
        #return content,title,published_date
    except AttributeError:
        try:
            response = requests.get(url)
            time.sleep(3)
            soup = BeautifulSoup(response.content, 'html5lib')
            content = get_content(url)
            post_time = soup.find('div', class_ = 'magazine_event_date').text.strip()
            published_date = convert_string(post_time)
            title = soup.find('div', class_ = 'titZing').text.strip()
        except AttributeError as e:
            print(e)
            content = ''
            title = ''
            published_date= ''
    return content,title,published_date

def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time() - 1*24*3600)
    for i in urls:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html5lib')
        try:
            date_posted = soup.find('time').text.strip()
            date_posted_norm = convert_time_string(date_posted)
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                print(i)
        except AttributeError as e:
            print(e)
            break
    return filtered_urls
def get_list_url(cat_url):
    urls = []
    response = requests.get(cat_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,'html5lib')
    try:
        news_block  = soup.find('section', id = 'tin_bai_noi_bat_khac')
        try:
            url_list = news_block.find_all('a')
            #print(len(url_list))
            for url in url_list:
                if 'https://www.24h.com.vn/' in url['href']:
                    if url['href'] not in urls:
                        urls.append(url['href'])
            print(len(urls))
        except AttributeError as e:
            print(e)
    except AttributeError as e:
        print(e)
    try:
        news_block  = soup.find('div', class_ = 'cate-24h-foot-home-latest-list')
        try:
            url_list = news_block.find_all('a')
            #print(len(url_list))
            for url in url_list:
                if 'https://www.24h.com.vn/' in url['href']:
                    if url['href'] not in urls:
                        urls.append(url['href'])
            print(len(urls))
        except AttributeError as e:
            print(e)
    except AttributeError as e:
        print(e)
    return urls
def add_list(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            
            urls = get_list_url(web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)

def add_post(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            web_24h_com_vn['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_24h_com_vn['urls'][i]['cate_id'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                

def get_news():
    web_24h_com_vn = {
        "home_page":"https://www.24h.com.vn/",
        "urls":{
            "tech":
            {
             "url":"https://www.24h.com.vn/cong-nghe-thong-tin-c55.html",
             "cate_id":57,
             "sub-category":{
                0:{"name":"Game",
                 "url":"https://www.24h.com.vn/game-c69.html"},
                1:{"name":"Phần mềm",
                 "url":"https://www.24h.com.vn/phan-mem-ngoai-c302.html"},
                2:{"name":"Khoa học",
                 "url":"https://www.24h.com.vn/khoa-hoc-c782.html"},
                3:{"name":"Mạng xã hội",
                 "url":"https://www.24h.com.vn/mang-xa-hoi-c889.html"},
                4:{"name":"Thủ thuật - Tiện ích"
                 ,"url":"https://www.24h.com.vn/thu-thuat-tien-ich-c68.html"},
                5:{"name":"Sợ Virus",
                 "url":"https://www.24h.com.vn/tim-hieu-virus-c57.html"},
                6:{"name":"Máy in/phụ kiện",
                 "url":"https://www.24h.com.vn/may-in/phu-kien-c291.html"},
                7:{"name":"Khám phá công nghệ",
                 "url":"https://www.24h.com.vn/kham-pha-cong-nghe-c675.html"}
             }
            }
            ,
            "youths":
            {
            "url":"https://www.24h.com.vn/ban-tre-cuoc-song-c64.html",
            "cate_id":60,
             "sub-category":{
                0:{"name":"Chuyện công sở","url":"https://www.24h.com.vn/chuyen-cong-so-c180.html"},
                1:{"name":"Tình yêu - Giới Tính","url":"https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html"},
                2:{"name":"Ngoại tình","url":"https://www.24h.com.vn/ngoai-tinh-c435.html"},
                3:{"name":"Giới trẻ","url":"https://www.24h.com.vn/gioi-tre-c434.html"},
                4:{"name":"Hotgirl - Hotboy","url":"https://www.24h.com.vn/hotgirl-hot-boy-c64e3398.html"},
                5:{"name":"Nhịp sống trẻ","url":"https://www.24h.com.vn/nhip-song-tre-c685.html"}
             }
            }
            ,
            "showbiz":
            {
            "cate_id":59,
            "url":"https://www.24h.com.vn/doi-song-showbiz-c729.html",
             "sub-category":{
                0:{"name":"Sao Việt","url":"https://www.24h.com.vn/sao-viet-c757.html"},
                1:{"name":"24h gặp gỡ","url":"https://www.24h.com.vn/gap-go-24h-c729e6820.html"},
                2:{"name":"Talk với sao","url":"https://www.24h.com.vn/doi-thoai-cung-sao-c730.html"},
                3:{"name":"Sao châu Á","url":"https://www.24h.com.vn/sao-chau-a-c759.html"},
            }
            },
            "cars":
            {
            "cate_id":58,
            "url":"https://www.24h.com.vn/o-to-c747.html",
             "sub-category":{
                0:{"name":"Tin tức ô tô","url":"https://www.24h.com.vn/tin-tuc-o-to-c332.html"},
                1:{"name":"Bảng giá xe ô tô","url":"https://www.24h.com.vn/bang-gia-xe-o-to-c807.html"},
                2:{"name":"Tư vấn","url":"https://www.24h.com.vn/tu-van-c240.html"},
                3:{"name":"Ngắm xe","url":"https://www.24h.com.vn/anh-nguoi-dep-va-xe-c199.html"},
                4:{"name":"Đánh giá xe","url":"https://www.24h.com.vn/so-sanh-xe-c805.html"},
            }
            }
        }
    }
#
    add_list(web_24h_com_vn)
    add_post(web_24h_com_vn)
    return web_24h_com_vn
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
        
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    web_24h_com_vn = get_news()
    for i in list(web_24h_com_vn['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(web_24h_com_vn['urls'][i]['sub-category']):
            url_list =  web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                text = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['text']
                title = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = web_24h_com_vn['urls'][i]['cate_id']
                print("title: ", title, "\n")
                print("date: ", published_date, "\n")
                print("id: ", cate_id, "\n")
                try:
                    text_len = len(text.text)
                    if text_len <300:
                        continue
                    else:
                         send_post_to_5goals(title,str(text),cate_id,published_date)
                except (AttributeError,TypeError):
                    continue
                
                """
                print(len(text.text))
                #print(url_list[t],title, cate_id, published_date,text)
                #return web_24h_com_vn"""



In [13]:
main()

11
19
tech 0 https://www.24h.com.vn/game-c69.html
https://www.24h.com.vn/cong-nghe-thong-tin/hang-ti-thiet-bi-co-ket-noi-bluetooth-nguy-co-bi-xam-nhap-ke-ca-iphone-c55a1524128.html
https://www.24h.com.vn/cong-nghe-thong-tin/danh-sach-15-ung-dung-mua-sam-thu-thap-du-lieu-nguoi-dung-nhieu-nhat-c55a1524028.html
11
19
tech 1 https://www.24h.com.vn/phan-mem-ngoai-c302.html
https://www.24h.com.vn/cong-nghe-thong-tin/bi-an-vat-the-ma-quai-moi-kinh-thien-van-thay-mot-chan-dung-khac-c55a1524076.html
11
19
tech 2 https://www.24h.com.vn/khoa-hoc-c782.html
11
19
tech 3 https://www.24h.com.vn/mang-xa-hoi-c889.html
11
19
tech 4 https://www.24h.com.vn/thu-thuat-tien-ich-c68.html
11
19
tech 5 https://www.24h.com.vn/tim-hieu-virus-c57.html
11
19
tech 6 https://www.24h.com.vn/may-in/phu-kien-c291.html
https://www.24h.com.vn/cong-nghe-thong-tin/nhung-ung-dung-chan-cuoc-goi-rac-duoc-nhieu-nguoi-su-dung-c55a1524017.html
11
19
tech 7 https://www.24h.com.vn/kham-pha-cong-nghe-c675.html
https://www.24h.com.vn

In [23]:
get_content('https://www.24h.com.vn/ban-tre-cuoc-song/co-dau-thanh-hoa-deo-28-cay-vang-trong-ngay-cuoi-chia-se-ve-cuoc-hon-nhan-song-gio-c64a1523648.html')

<article class="cate-24h-foot-arti-deta-info"><h2>Quỳnh Như từng “gây bão” mạng xã hội với loạt ảnh đeo vàng trĩu cổ, trong đó có những chiếc kiềng vàng được thiết kế tinh xảo.</h2><p><div>[caption id="" align="aligncenter" width="800"]</div><img alt="Cô dâu Thanh Hóa đeo 28 cây vàng trong ngày cưới chia sẻ về cuộc hôn nhân sóng gió - 1" class="aligncenter" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-02/anh-1-1701484407-560-width660height495.jpg" width="800"/>Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới[/caption]</p><p>Đầu năm 2022, khi dân mạng chưa hết ngỡ ngàng bởi một đám cưới ngập vàng ở Nghệ An thì lại tiếp tục “choáng ngợp” bởi hình ảnh cô dâu Thanh Hóa đeo vàng chi chít cổ. Cô dâu sở hữu nhan sắc xinh đẹp, đeo vàng chồng chéo trên cổ, cổ tay và ngón tay, trong đó có những chiếc kiềng vàng lấp lánh, được thiết kế cầu kỳ, tinh xảo.</p><p>Cô dâu đó là Quỳnh Như, quê Thanh Hóa, đám cưới được tổ chức vào tháng 1/2022. Quỳnh Như cho hay, vợ chồng 

In [201]:
len('Ngoại hình của cô gái gây ấn tượng mạnh với chàng trai này, thậm chí, anh xem đây là người vợ trong mơ mình ao ước. Ở một số vùng nông thôn Trung Quốc, nhiều người trẻ tuổi vẫn chọn cách mai mối để tìm bạn đời. Vì hai bên chưa từng gặp nhau trước khi xem mắt nên thành công hay thất bại phụ thuộc phần lớn vào sự may mắn.')

321

In [56]:
def get_content(url):
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,  'html.parser')
    try:
        #remove the last 4 element
        article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
        #h2 = article.find('h2')
        #h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
        # Update the text of the h2 tag
        #h2.string = h2_text
        for script_or_style in article(['script', 'style']):
            script_or_style.decompose()
        article.find('div',recursive = True, class_ = 'bv-lq').decompose()
        article.find('p', class_ = 'linkOrigin').decompose()
        caption_text_list = []
        caption_list =article.find_all('p',class_='img_chu_thich_0407')
        for i in caption_list:
            caption_text_list.append(i.text.strip())
        
        #find a,span and remove tag with text
        tags_to_remove = article.find_all(['a', 'span'])
        for tag in tags_to_remove:
            if tag.text !=  '':
                # Extract the text from the tag
                tag_text = tag.get_text()
                # Replace the tag with its text content
                tag.replace_with(tag_text)
                tag.text.strip()
        # set attribut for img
        b = article.find_all('img')
        for i in b:
            if '.svg' in i['src']:
                i.decompose()
        for i in article.find_all('img'):
            if 'data-original' in i.attrs.keys():
                i['src'] = i.get('data-original')
            #i.attrs = ['class', 'alt', 'src', 'data-original']
            #del i['onclick']
            #del i['style']
            #del i['class']
        list_attr = ['src','alt','data-original']
        for i in article.find_all('img'):
            for j in list(i.attrs.keys()):
                if j not in list_attr:
                    i.attrs.pop(j)
        for i in article.find_all('img'):
            if 'https://image-us.24h.com.vn/' not in i['src']:
                i['src']= i['data-original']
        img_list = article.find_all('img')
        n_img = len(img_list)
        #print(len(caption_text_list))
        for i in range(0,n_img):
            try:
                # Create NavigableString objects for the custom caption tags
                caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
                caption_end = NavigableString("[/caption]")
                try:
                    caption_text = NavigableString(caption_text_list[i])
                except IndexError:
                    caption_text = ''
                # Insert the caption start tag before the <img> tag
                img_list[i].insert_before(caption_start)
                # Insert the caption end tag after the <img> tag
                img_list[i].insert_after(caption_end)
                img_list[i].insert_after(caption_text)
                # Wrap the entire structure (caption start, img, caption end) in a new tag if needed
                # For example, wrapping in a <div>
                new_div = soup.new_tag("div")
                caption_start.wrap(new_div)
                #caption_text_list[i].decompose()
            except IndexError as e:
                print(e)
        for i in article.find_all(recursive = True):
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
                
            except TypeError:
                continue
        for i in article.find_all('img'):
            i['class'] = "aligncenter"
            i['width'] = 800
            i['height'] = 400
        source_tag = soup.new_tag('i') 
        source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
        #remove empty div or empty space from element
        """for i in article.find_all('div',recursive= True):
            if i.string == None:
                if i.text =='':
                    i.decompose()"""
        for element in article.find_all(recursive = True,string=True):
            if isinstance(element, NavigableString) and element.strip() == '':
                element.extract()
        #remove html comment from element   
        for comment in article.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()
    # Append the <i> tag as the last child of the <article> tag
        article.append(source_tag)
    except AttributeError as e:
        print(e)
    return article

In [57]:
get_content('https://www.24h.com.vn/ban-tre-cuoc-song/co-dau-thanh-hoa-deo-28-cay-vang-trong-ngay-cuoi-chia-se-ve-cuoc-hon-nhan-song-gio-c64a1523648.html')

<article class="cate-24h-foot-arti-deta-info"><h2><strong>
                    Quỳnh Như từng “gây bão” mạng xã hội với loạt ảnh đeo vàng trĩu cổ, trong đó có những chiếc kiềng vàng được thiết kế tinh xảo.                </strong></h2><p><div>[caption id="" align="aligncenter" width="800"]</div><img alt="Cô dâu Thanh Hóa đeo 28 cây vàng trong ngày cưới chia sẻ về cuộc hôn nhân sóng gió - 1" class="aligncenter" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-02/anh-1-1701484407-560-width660height495.jpg" width="800"/>Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới[/caption]</p><p>Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới</p><p>Đầu năm 2022, khi dân mạng chưa hết ngỡ ngàng bởi một đám cưới ngập vàng ở Nghệ An thì lại tiếp tục “choáng ngợp” bởi hình ảnh cô dâu Thanh Hóa đeo vàng chi chít cổ. Cô dâu sở hữu nhan sắc xinh đẹp, đeo vàng chồng chéo trên cổ, cổ tay và ngón tay, trong đó có những chiếc kiềng vàng lấp lánh, được thiết kế cầu kỳ, tinh xảo.</p

In [64]:
response = requests.get('https://www.24h.com.vn/ban-tre-cuoc-song/co-dau-thanh-hoa-deo-28-cay-vang-trong-ngay-cuoi-chia-se-ve-cuoc-hon-nhan-song-gio-c64a1523648.html')
soup = BeautifulSoup(response.content,  'html.parser')
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
caption_text_list =article.find_all('p',class_='img_chu_thich_0407', recursive = False)
for i in caption_text_list:
    print(i.text.strip())
#caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
len(caption_text_list)

Cô dâu Thanh Hóa đeo chi chít vàng trong ngày cưới
Cô dâu gây chú ý bởi nhan sắc xinh đẹp
Cặp đôi có 5 năm bên nhau trước khi về chung một nhà
Cặp đôi từng trải qua biến cố lớn trong hôn nhân

Vợ chồng Quỳnh Như đã "gương vỡ lại lành"


Cô dâu Thanh Hóa ngày càng xinh đẹp.


9

In [60]:
article.find_all('img')

[<img alt="Cô dâu Thanh Hóa đeo 28 cây vàng trong ngày cưới chia sẻ về cuộc hôn nhân sóng gió - 1" class="news-image" onclick="show_slide_image_news(0)" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-02/anh-1-1701484407-560-width660height495.jpg" style="width: 660px; height: 495px;"/>,
 <img alt="Cô dâu Thanh Hóa đeo 28 cây vàng trong ngày cưới chia sẻ về cuộc hôn nhân sóng gió - 2" class="news-image" data-original="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-02/anh-2-1701484437-193-width660height914.jpg" onclick="show_slide_image_news(1)" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABAQMAAAAl21bKAAAAA1BMVEX6+vqsEtnpAAAACklEQVQI12NgAAAAAgAB4iG8MwAAAABJRU5ErkJggg==" style="width: 660px; height: 914px;"/>,
 <img alt="Cô dâu Thanh Hóa đeo 28 cây vàng trong ngày cưới chia sẻ về cuộc hôn nhân sóng gió - 3" class="news-image" data-original="https://image-us.24h.com.vn/upload/4-2023/images/2023-12-02/anh-3-1701484453-677-width660height440.jpg" onclick=

In [63]:
caption_text_list = soup.find_all('p',class_='img_chu_thich_0407',recursive = False)
for i in caption_text_list:
   print(i.text.strip())

In [38]:
soup.find_all('p',class_='img_chu_thich_0407')

[<p class="img_chu_thich_0407" style="text-align: center;">Nhiều cặp đôi đang mải miết với smartphone hơn thay vì tập trung quan tâm đến nửa bên kia.</p>,
 <p class="img_chu_thich_0407" style="text-align: center;">Sự xao lãng trong tình yêu sẽ dễ khiến các cặp đôi tan vỡ.</p>]

In [99]:
for i in article.find_all(recursive = True):
    try:
        i.string = i.text.strip()
    except AttributeError:
        continue


In [112]:
for i in article.find_all('p', recursive = True):
    if i.get('style') == 'text-align: center;':
        print(i)

<p style="text-align: center;"><em>Cậu bé có nhiều nét tương đồng, luôn đứng cạnh Ngọc Sơn.</em></p>
<p style="text-align: center;"><em>Ngọc Sơn vẫn hoạt động nghệ thuật năng nổ ở tuổi 55. </em></p>
<p style="text-align: center;"><em>Nhà từ đường được Ngọc Sơn chi tiền xây dựng cũng như thiết kế, sắm sửa nội thất.</em></p>


In [219]:
for i in list(web_24h_com_vn['urls'].keys()):
#web_24h_com_vn2['url'][i]['cate_id']
    for j in list(web_24h_com_vn['urls'][i]['sub-category']):
        url =  web_24h_com_vn['urls'][i]['sub-category'][j]['url']
        list_url = get_list_url(url)
        content,title,published_date = get_post(list_url[0])
        print(i,j)
        print(title,'\n_______________\n',published_date,'\n___________________ \n')
        

11
19
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
tech 0
"Bỏng mắt" trước hình ảnh do AI vẽ nữ nhân vật Yelan game Genshin Impact 
_______________
 2023-11-02 
___________________ 

11
19
tech 1
Cảnh sát đề nghị phụ huynh nên tắt tính năng mới này trên iPhone của trẻ em 
_______________
 2023-11-28 
___________________ 

11
19
tech 2
NASA bắt được dấu hiệu sinh học trên hành tinh 1,5 tỉ tuổi 
_______________
 2023-11-27 
___________________ 

11
19
tech 3
Người dùng mạng xã hội bị tận thu, lợi dụng? 
_______________
 2023-11-20 
___________________ 

11
19
tech 4
Cách trò chuyện với ChatGPT bằng giọng nói miễn phí 
_______________
 2023-11-24 
___________________ 

11
19
tech 5
Tại sao năm 2024 sẽ là một năm đầy thách thức với an ninh mạng? 
_______________
 2023-11-24 
___________________ 

11
19
tech 6
Sony giới th

In [84]:
img_list = article.find_all('img')
n_img = len(img_list)
caption_text_list =article.find_all('p',class_='img_chu_thich_0407')

#print(len(caption_text_list))
for i in range(0,n_img):
    try:
        """caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption_text_list[i].get_text())
        caption_end = NavigableString("[/caption]")
        caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
        img_list[i].insert_before(caption_start)
        img_list[i].insert_after(caption_end)
        img_list[i].insert_after(caption_text) """
        # Create NavigableString objects for the custom caption tags
        caption_start = NavigableString("[caption]")
        caption_end = NavigableString("[/caption]")
        try:
            caption_text = NavigableString(caption_text_list[i].get_text())
        except IndexError:
            caption_text = ''
        # Insert the caption start tag before the <img> tag
        img_list[i].insert_before(caption_start)
        # Insert the caption end tag after the <img> tag
        img_list[i].insert_after(caption_end)
        img_list[i].insert_after(caption_text)
        # Wrap the entire structure (caption start, img, caption end) in a new tag if needed
        # For example, wrapping in a <div>
        new_div = soup.new_tag("div")
        caption_start.wrap(new_div)
    except IndexError as e:
        continue

In [137]:
web_24h_com_vn = get_news()
web_24h_com_vn

11
19
tech 0 https://www.24h.com.vn/game-c69.html
11
19
tech 1 https://www.24h.com.vn/phan-mem-ngoai-c302.html
https://www.24h.com.vn/cong-nghe-thong-tin/canh-sat-de-nghi-phu-huynh-nen-tat-tinh-nang-moi-nay-tren-iphone-cua-tre-em-c55a1522505.html
https://www.24h.com.vn/cong-nghe-thong-tin/nhieu-nguoi-dung-phan-nan-google-drive-bi-mat-du-lieu-c55a1522491.html
https://www.24h.com.vn/cong-nghe-thong-tin/ios-1712-san-sang-ra-mat-co-gi-dang-cho-doi-c55a1522411.html
11
19
tech 2 https://www.24h.com.vn/khoa-hoc-c782.html
11
19
tech 3 https://www.24h.com.vn/mang-xa-hoi-c889.html
11
19
tech 4 https://www.24h.com.vn/thu-thuat-tien-ich-c68.html
11
19
tech 5 https://www.24h.com.vn/tim-hieu-virus-c57.html
11
19
tech 6 https://www.24h.com.vn/may-in/phu-kien-c291.html
11
19
tech 7 https://www.24h.com.vn/kham-pha-cong-nghe-c675.html
https://www.24h.com.vn/kham-pha-cong-nghe/skycomputer-dia-chi-mua-sam-va-sua-chua-may-tinh-uy-tin-dang-tin-cay-c675a1522209.html
14
20
youths 0 https://www.24h.com.vn/chuy

{'home_page': 'https://www.24h.com.vn/',
 'urls': {'tech': {'url': 'https://www.24h.com.vn/cong-nghe-thong-tin-c55.html',
   'cate_id': 57,
   'sub-category': {0: {'name': 'Game',
     'url': 'https://www.24h.com.vn/game-c69.html',
     'url_list': [],
     'content': {}},
    1: {'name': 'Phần mềm',
     'url': 'https://www.24h.com.vn/phan-mem-ngoai-c302.html',
     'url_list': ['https://www.24h.com.vn/cong-nghe-thong-tin/canh-sat-de-nghi-phu-huynh-nen-tat-tinh-nang-moi-nay-tren-iphone-cua-tre-em-c55a1522505.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/nhieu-nguoi-dung-phan-nan-google-drive-bi-mat-du-lieu-c55a1522491.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/ios-1712-san-sang-ra-mat-co-gi-dang-cho-doi-c55a1522411.html'],
     'content': {0: {'text': <article class="cate-24h-foot-arti-deta-info">
       <h2>
       <strong>
                           Một tính năng mới trên iOS 17 đang được đề nghị vô hiệu hóa trên các iPhone của trẻ nhỏ.                </stro

In [138]:
web_24h_com_vn

{'home_page': 'https://www.24h.com.vn/',
 'urls': {'tech': {'url': 'https://www.24h.com.vn/cong-nghe-thong-tin-c55.html',
   'cate_id': 57,
   'sub-category': {0: {'name': 'Game',
     'url': 'https://www.24h.com.vn/game-c69.html',
     'url_list': [],
     'content': {}},
    1: {'name': 'Phần mềm',
     'url': 'https://www.24h.com.vn/phan-mem-ngoai-c302.html',
     'url_list': ['https://www.24h.com.vn/cong-nghe-thong-tin/canh-sat-de-nghi-phu-huynh-nen-tat-tinh-nang-moi-nay-tren-iphone-cua-tre-em-c55a1522505.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/nhieu-nguoi-dung-phan-nan-google-drive-bi-mat-du-lieu-c55a1522491.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/ios-1712-san-sang-ra-mat-co-gi-dang-cho-doi-c55a1522411.html'],
     'content': {0: {'text': <article class="cate-24h-foot-arti-deta-info">
       <h2>
       <strong>
                           Một tính năng mới trên iOS 17 đang được đề nghị vô hiệu hóa trên các iPhone của trẻ nhỏ.                </stro

In [155]:
for i in [list(web_24h_com_vn['urls'].keys())[0]]:
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(web_24h_com_vn['urls'][i]['sub-category']):
            url_list =  web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                text = str(web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['text'])
                title = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = web_24h_com_vn['urls'][i]['cate_id']
                send_post_to_5goals(title,text,str(cate_id),published_date)
                #send_post_to_5goals(title,content,category_id,published_date)
                #print("url: ", url_list[t], "\n")
                #print("title: ", title, "\n")
                #print("date: ", published_date, "\n")
                #print("id: ", cate_id, "\n")
                #print("text: ", str(text), "\n")
                

The post was successfully created.
Response: {"success":true,"post_id":64528}
text:  <article class="cate-24h-foot-arti-deta-info">
<h2>
<strong>
                    Một tính năng mới trên iOS 17 đang được đề nghị vô hiệu hóa trên các iPhone của trẻ nhỏ.                </strong>
</h2>
 <p>Theo <em>PhoneArena</em>, một sở cảnh sát địa phương ở tiểu bang Ohio (Mỹ) đang cảnh báo các bậc phụ huynh về một tính năng mới được bật theo mặc định trên iPhone sau khi cập nhật lên iOS 17.1. Được gọi là NameDrop, tính năng này cho phép hai chiếc iPhone có thể trao đổi nhanh thông tin liên lạc gồm số điện thoại và hình ảnh bằng cách đặt chúng gần nhau.</p>
<p align="center"></p><div>
<div>
<div><section><div>
</div>
</section><div>[caption id="" align="aligncenter" width="800"]</div><img alt="Cảnh sát đề nghị phụ huynh nên tắt tính năng mới này trên iPhone của trẻ em - 1" class="aligncenter" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-28/picture-2075212531-1701145775-4

In [84]:
"Lam Trường: &#34;Nhiều người hỏi vì sao Yến Phương là vợ của tôi mà còn phải bán hàng online&#34; - 1"[:-4]

'Lam Trường: &#34;Nhiều người hỏi vì sao Yến Phương là vợ của tôi mà còn phải bán hàng online&#34;'

In [160]:
list(web_24h_com_vn['urls'].keys())[0]

'tech'

In [175]:
web_24h_com_vn

{'home_page': 'https://www.24h.com.vn/',
 'urls': {'tech': {'url': 'https://www.24h.com.vn/cong-nghe-thong-tin-c55.html',
   'cate_id': 57,
   'sub-category': {0: {'name': 'Game',
     'url': 'https://www.24h.com.vn/game-c69.html',
     'url_list': [],
     'content': {}},
    1: {'name': 'Phần mềm',
     'url': 'https://www.24h.com.vn/phan-mem-ngoai-c302.html',
     'url_list': ['https://www.24h.com.vn/cong-nghe-thong-tin/ios-18-va-nhung-tinh-nang-duoc-don-thoi-se-cap-ben-iphone-c55a1522292.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/chatgpt-xuat-hien-khap-noi-vua-duoc-nhung-trong-mot-tro-ly-ai-moi-c55a1522186.html',
      'https://www.24h.com.vn/cong-nghe-thong-tin/doi-thu-cua-chatgpt-thong-minh-tung-ngay-c55a1522122.html'],
     'content': {0: {'text': <article class="cate-24h-foot-arti-deta-info">
       <h2>
       <strong>
                           Vẫn còn một thời gian dài trước khi iOS 18 được công bố, nhưng đã có nhiều tính năng tin đồn của hệ điều hành được lan 

In [48]:
def get_content(url):
    response = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(response.content,  "html5lib")
    try:
        #remove the last 4 element
        article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
        for script_or_style in article(['script', 'style']):
            script_or_style.decompose()
        children = article.find_all(recursive=False)
        for i in children[-4:]:
            i.decompose()
        caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
        #find a,span and remove tag with text
        tags_to_remove = article.find_all(['a', 'span'])
        for tag in tags_to_remove:
            if tag.text !=  '':
                # Extract the text from the tag
                tag_text = tag.get_text()
                # Replace the tag with its text content
                tag.replace_with(tag_text)
                tag.text.strip()
        # set attribut for img
        b = article.find_all('img')
        for i in b:
            if '.svg' in i['src']:
                i.decompose()
        for i in article.find_all('img'):
            if 'data-original' in i.attrs.keys():
                i['src'] = i.get('data-original')
            #i.attrs = ['class', 'alt', 'src', 'data-original']
            del i['onclick']
            del i['style']
            del i['class']
        img_list = article.find_all('img')
        n_img = len(img_list)
        #print(len(caption_text_list))
        for i in range(0,n_img):
            try:
                # Create NavigableString objects for the custom caption tags
                caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
                caption_end = NavigableString("[/caption]")
                try:
                    caption_text = NavigableString(caption_text_list[i].get_text())
                except IndexError:
                    caption_text = ''
                # Insert the caption start tag before the <img> tag
                img_list[i].insert_before(caption_start)
                # Insert the caption end tag after the <img> tag
                img_list[i].insert_after(caption_end)
                img_list[i].insert_after(caption_text)
                # Wrap the entire structure (caption start, img, caption end) in a new tag if needed
                # For example, wrapping in a <div>
                new_div = soup.new_tag("div")
                caption_start.wrap(new_div)
                caption_text_list[i].decompose()
            except IndexError as e:
                continue
        for i in article.find_all(recursive = True):
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
                del i['href']
                i.text.strip()
            except AttributeError:
                continue
            #try:
                #print(i)
                
            except TypeError:
                continue
        for i in article.find_all('img'):
            i['class'] = "aligncenter"
            i['width'] = 800
            i['height'] = 400
        for video in article.find_all('video'):
            video.decompose()
        for i in article.find_all('div', {'align': 'center'}):
            i.decompose()
        for item in article.find_all('div'):
            if item.string =="":
                item.decompose()
        for element in article.find_all(recursive = True,string=True):
            if isinstance(element, NavigableString) and element.strip() == '':
                element.extract()
        for i in article.find_all('div'):
            if i.children == None and i.string == None:
                i.decompose()
        source_tag = soup.new_tag('i') 
        source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
        article.find('h2').text.strip()
    # Append the <i> tag as the last child of the <article> tag
        article.append(source_tag)
        
    except AttributeError as e:
        print(e)
    return article

In [49]:
get_content('https://www.24h.com.vn/doi-song-showbiz/hoc-tro-sieu-mau-ha-anh-hoan-thanh-ca-mo-6-tieng-de-lay-lai-giong-noi-c729a1523228.html')

<article class="cate-24h-foot-arti-deta-info"><h2><strong>
                    Quang Sơn vừa trải qua hành trình phẫu thuật để lấy lại giọng nói nhằm tiếp tục hoạt động nghệ thuật.                </strong></h2><p>Quang Sơn là diễn viên trẻ được biết đến với nhiều vai phản diện, giang hồ. Anh được biết đến với những vai diễn  trong loạt phim <em>Cú Đấm Thép, Chị Mười Ba, Ông Trùm</em>,... Nam diễn viên sinh năm 1996 tốt nghiệp Trường đại học Sân khấu Điện ảnh TP.HCM, từng giành ngôi vị quán quân <em>Gương Mặt Điện Ảnh 2019</em>.</p><p></p><div><a><div>[caption id="" align="aligncenter" width="800"]</div><img alt="Học trò siêu mẫu Hà Anh hoàn thành ca mổ 6 tiếng để lấy lại giọng nói - 1" class="aligncenter" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-30/336298333_1247725819497014_1341452322055725607_n-1701336544-500-width2048height1366.jpg" width="800"/>Quang Sơn là diễn viên "có nghề", được nhiều đạo diễn yêu thích.[/caption]</a><a><div>[caption id="" alig